In [3]:
%matplotlib inline
#This helps widgets somehow

In [2]:
! pip install -Uq fastai fastbook setuptools ipywidgets RandomWords gradio

In [ ]:
#hide
import fastbook
fastbook.setup_book()

In [5]:
#hide
from fastbook import *
from fastai.vision.widgets import *

In [6]:
from pathlib import Path
modelFolder = Path.cwd().parent.parent / 'Trained-models'
# If you want a spicific model version, then add /'version number' or 'latest' to the path

In [7]:
import os

def find_files_with_extension(file_extension):
    current_dir = os.getcwd()
    matching_files = [file for file in os.listdir(modelFolder) if file.endswith(file_extension)]
    return matching_files

# Example usage
file_type = '.pkl'  # Change this to the file type you're looking for
matching_files = find_files_with_extension(file_type)

if matching_files:
    print(f"This directory contains the following '{file_type}' file(s):")
    for file in matching_files:
        print(f"- {file}")
else:
    print(f"This directory does not contain any '{file_type}' files")

This directory contains the following '.pkl' file(s):
- Animal Classifier.pkl
- league_champion_classifier.pkl
- test.pkl


In [23]:
modelName = 'Animal Classifier.pkl'
modelPath = f'{modelFolder} / {modelName}'
print(f"Path: '{modelFolder}'")

Path: 'c:\Users\Gamer\Documents\GitHub\VS-Code\Side-Projects\Trained-models'


In [38]:
if(os.system('uname -o') == 1):
    modelPath = Path.path(f'{modelPath}').as_windows()

learn_inf = load_learner(modelPath)
print("Model loaded successfully!")

AttributeError: type object 'Path' has no attribute 'path'

In [ ]:
#hide_output
btn_upload = widgets.FileUpload()
out_pl = widgets.Output()
lbl_pred = widgets.Label()
btn_run = widgets.Button(description='Classify')

In [ ]:
def on_click_classify(change):
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(448,448))
    pred,pred_idx,probs = learn_inf.predict(img)
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.02f}'

btn_run.on_click(on_click_classify)

In [ ]:
VBox([btn_upload, btn_run, out_pl, lbl_pred])

In [ ]:
import gradio as gr
from fastai.vision.all import *
import skimage

learn = learn_inf
labels = learn.dls.vocab
def predict(img):
    img = PILImage.create(img)
    pred,pred_idx,probs = learn.predict(img)
    return {labels[i]: float(probs[i]) for i in range(len(labels))}

interface = gr.Interface(
    fn=predict,
    inputs=gr.Image(type="pil"),
    outputs=gr.Label(num_top_classes=10),
    title=model_name,
    description="")

interface.launch(share=True)